# Logistic Regression - Women Risk Predictor

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix, roc_curve
import joblib
import os

In [ ]:
fp = pd.read_csv("../data/women_risk_processed.csv")

In [ ]:
fp.head()

In [ ]:
fp.shape

In [ ]:
target_col = '12. Overall, how would you rate the risk level of harassment in that situation?'
X = fp.drop(target_col, axis=1)
y = fp[target_col]

y = (y > y.median()).astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
model = LogisticRegression(max_iter=1000, random_state=42)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
precision_score(y_test, y_pred)

In [ ]:
recall_score(y_test, y_pred)

In [ ]:
f1_score(y_test, y_pred)

In [ ]:
roc_auc_score(y_test, y_pred_proba)

In [ ]:
classification_report(y_test, y_pred)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix - Logistic Regression')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = roc_auc_score(y_test, y_pred_proba)

plt.figure(figsize=(6, 4))
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Logistic Regression')
plt.legend()
plt.show()

## Hyperparameter Tuning

In [ ]:
param_grid = {
    "C": [0.01, 0.1, 1, 10, 100],
    "penalty": ['l2'],
    "solver": ['lbfgs', 'liblinear']
}

In [ ]:
grid = GridSearchCV(LogisticRegression(max_iter=1000, random_state=42), param_grid, cv=5, scoring='accuracy', n_jobs=-1)

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
print("Best parameters:", grid.best_params_)
print("Best CV accuracy:", grid.best_score_)

In [ ]:
best_model = grid.best_estimator_

In [ ]:
y_pred_tuned = best_model.predict(X_test)
y_pred_proba_tuned = best_model.predict_proba(X_test)[:, 1]

In [ ]:
accuracy_score(y_test, y_pred_tuned)

In [ ]:
classification_report(y_test, y_pred_tuned)

In [ ]:
cm_tuned = confusion_matrix(y_test, y_pred_tuned)
plt.figure(figsize=(6, 4))
sns.heatmap(cm_tuned, annot=True, fmt='d', cmap='Greens')
plt.title('Confusion Matrix - Tuned Model')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']
before = [accuracy_score(y_test, y_pred), precision_score(y_test, y_pred), 
          recall_score(y_test, y_pred), f1_score(y_test, y_pred), roc_auc_score(y_test, y_pred_proba)]
after = [accuracy_score(y_test, y_pred_tuned), precision_score(y_test, y_pred_tuned), 
         recall_score(y_test, y_pred_tuned), f1_score(y_test, y_pred_tuned), roc_auc_score(y_test, y_pred_proba_tuned)]

x = np.arange(len(metrics))
width = 0.35

plt.figure(figsize=(10, 5))
plt.bar(x - width/2, before, width, label='Before Tuning')
plt.bar(x + width/2, after, width, label='After Tuning')
plt.xlabel('Metrics')
plt.ylabel('Score')
plt.title('Model Performance Comparison')
plt.xticks(x, metrics)
plt.legend()
plt.ylim([0, 1.1])
plt.show()

## Save Model

In [ ]:
os.makedirs('../models', exist_ok=True)
joblib.dump(best_model, '../models/logistic_regression_model.pkl')
print("Model saved!")